#A. Descargar y descomprimir los datos

In [ ]:
# Descargar el archivo profner.zip desde Zenodo
!wget https://zenodo.org/record/4563995/files/profner.zip?download=1

# Descomprimir el archivo descargado
!unzip profner.zip?download=1

#B. Funciones de carga y preparación de datos

In [2]:
import sys
import os
import pandas as pd
def get_tweet_content(list_paths):
  """
    Función para guardar en un diccionario el contenido de archivos txt
    que se introduce en su entrada.
    Devuelve un diccionario en el que las claves son el id del tweet, y
    el valor el texto del tweet.
  """
  output_dict = dict()
  for i in list_paths:
    tweet_id = i.split("/")[-1].split(".txt")[0]
    with open(i) as f:
      output_dict[int(tweet_id)] = f.read()

  return output_dict

def get_profner_data(profner_path_data):
    # Obtenemos el path a los txt de los tweets.
    path_to_txt = profner_path_data+"subtask-1/train-valid-txt-files/"
    tweets_train_files = [path_to_txt+"train/"+i for i in os.listdir(path_to_txt+"train/")]
    tweets_valid_files = [path_to_txt+"valid/"+i for i in os.listdir(path_to_txt+"valid/")]
    # Obtenemos diccionarios en los que el key es el tweet_id y el value el texto del tweet.
    train_txt_content = get_tweet_content(tweets_train_files)
    valid_txt_content = get_tweet_content(tweets_valid_files)

    # Cargamos dos dataframes con los tweet_id y la categoría de los tweets
    path_to_labeled = profner_path_data+"subtask-1/"
    train_tweets = pd.read_csv(path_to_labeled+"train.tsv",sep="\t")
    valid_tweets = pd.read_csv(path_to_labeled+"valid.tsv",sep="\t")

    # Introducimos a los df el campo de texto mapeando los diccionarios con tweet_id
    train_tweets["tweet_text"] = train_tweets['tweet_id'].map(train_txt_content)
    train_tweets["set"] = "train"
    valid_tweets["tweet_text"] = valid_tweets['tweet_id'].map(valid_txt_content)
    valid_tweets["set"] = "valid"

    # Concatenamos el resultado
    output_df = pd.concat([train_tweets,valid_tweets],axis=0)
    # Eliminamos retorno de carro
    output_df["tweet_text"] = output_df.tweet_text.apply(lambda x: x.replace('\n', ' '))
    return output_df[["tweet_id","tweet_text","label","set"]].reset_index(drop=True)

# Ejercicio


## 0. Imports


In [3]:
!pip install transformers


En este apartado voy a instalar y a importar todas las librerias que voyy a utilizar a lo largo del ejercicio, así como alguna descarga de archivos de estas:

In [4]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AutoModelForSequenceClassification,
   AdamW
)
import tensorflow as tf

In [5]:
import spacy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re

import scipy as sp
from sklearn.model_selection import train_test_split

import warnings

## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `get_profner_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [6]:
profner_path = "./profner/"
datos = get_profner_data(profner_path)

## 2. Pre-procesado

Vamos a usar el modelo ya preentrenado de hugging face:

https://huggingface.co/PlanTL-GOB-ES/roberta-large-bne

In [8]:
model_name = "PlanTL-GOB-ES/roberta-large-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Dividimos el conjunto de datos en Train-Validation (90%) y Test (10%).

In [9]:
train_texts, test_texts, train_labels, test_labels = train_test_split(datos["tweet_text"], datos["label"], test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Antes de preprocesar la entrada, necesitamos definir la longitud de la secuencia (longitud maxima del documento en tokens) y el Batch size. En Colab, hay limitaciones (batch = 8 y secuencia=96). Si se reduciera el batch se podría incrementar el tamaño, pero en este caso nos es indiferente.

In [10]:
max_seq_length = 96 #@param {type: "integer"}
train_batch_size =  8#@param {type: "integer"}
eval_batch_size = 8 #@param {type: "integer"}
test_batch_size = 8 #@param {type: "integer"}

Preprocesamos nuestros datos utilizando el tokenizador definido previamente:

In [11]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, max_length=max_seq_length, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, max_length=max_seq_length, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, max_length=max_seq_length, padding=True)

In [12]:
train_encodings[0]

Encoding(num_tokens=96, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

Se observa que el texto se sustituye por valores numéricos en un vector de tamaño ``max_seq_length´´

In [13]:
test_encodings

{'input_ids': [[0, 780, 5162, 362, 2876, 371, 1063, 341, 334, 799, 407, 25125, 320, 1300, 13787, 4288, 320, 332, 1742, 341, 2606, 1207, 341, 407, 3252, 320, 2425, 768, 365, 6689, 341, 14011, 344, 40355, 388, 332, 40255, 2429, 1632, 371, 1629, 85, 1068, 29269, 90, 392, 13993, 1068, 29269, 30895, 2181, 331, 4186, 1428, 138, 68, 473, 69, 31534, 75, 90, 128, 29489, 112, 45824, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1265, 32574, 80, 450, 28470, 313, 3740, 16104, 320, 341, 29018, 344, 1669, 313, 13673, 999, 6780, 2933, 26402, 10890, 2176, 4650, 26292, 7387, 4186, 1428, 138, 68, 473, 69, 110, 122, 76, 129, 90, 75, 31937, 134, 112, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 477, 2398, 313, 974, 9618, 957, 923, 355, 8042, 313, 2022, 2464, 313, 4993, 31604, 67, 2590, 3053, 394, 2439, 66, 342, 447, 15706, 1276,

Después generamos los tensores:

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

## 3. Entrenamiento

En este fine-tuning utilizando [TFAutoModelForSequenceClassification](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#tfautomodelforsequenceclassification) vamos a:



*   Especificar el número de clases de salida
*   Especificar el learning_rate, número de epocas.
*   Especificar optimizadores, Loss Functions y métricas de evaluación
*   Entrenar el modelo




In [15]:
# Numero de etiquetas
num_labels = 2
# Numero de muestras de entrenamiento y validación
num_train_examples = len(train_dataset)
num_dev_examples = len(val_dataset)
# Train steps
train_steps_per_epoch = int(num_train_examples / train_batch_size)
dev_steps_per_epoch = int(num_dev_examples / eval_batch_size)
# LEarning rate y epochs
learning_rate = 2e-5
num_epochs = 1
# Preparción del train y val dataset en batches con la configuración especificada.
train_dataset = train_dataset.shuffle(100).batch(train_batch_size)
val_dataset = val_dataset.shuffle(100).batch(eval_batch_size)

In [16]:
# Configuración del AutoModel para clasificación de secuencias (nuestro caso)
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config, from_pt=True, ignore_mismatched_sizes=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [17]:
# Building/compilación del modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Entrenamos el modelo compilado, que realmente será un fine-tuning:

In [ ]:
history = model.fit(train_dataset,
  epochs=num_epochs,
  steps_per_epoch=train_steps_per_epoch,
  validation_data=val_dataset,
  validation_steps=dev_steps_per_epoch)

 91/720 [==>...........................] - ETA: 6:43:29 - loss: 0.5576 - accuracy: 0.7541

Ha tardado 8 horas **(cuidado con darle a correr)** 😓

##4.  Test

Vamos a comprobar el resultado final del modelo. TAmbién vamos a hacer un classification report para ver FP y FN, y comprobar que las métricas en test son similares a las de development.

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset.batch(test_batch_size),verbose=2)
print('\nTest accuracy:', test_acc)

Hacemos predicción del test set con el modelo (cogemos logits) y argmax.


In [ ]:
y_pred = model.predict(test_dataset.batch(eval_batch_size)).logits
y_pred = np.argmax(y_pred,axis=1)

Las true labels son:

In [ ]:
y_true = [y for x,y in test_dataset]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true,y_pred))